建模过程：训练数据-特征提取-建立模型-预测-结果


In [1]:
# import libraries necessary for this project
import os, sys, pickle

import numpy as np
import pandas as pd

from datetime import date

from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve
from sklearn.preprocessing import MinMaxScaler

# display for this notebook
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


In [2]:
#读取三个表格的数据：线上训练集、线下训练集、线下测试集
dfoff = pd.read_csv('tianchi/ccf_offline_stage1_train.csv',keep_default_na = False)#必须设置keep_default_na=Falese，否则读取为NaN
dfon = pd.read_csv('tianchi/ccf_online_stage1_train.csv',keep_default_na = False)
dftest = pd.read_csv('tianchi/ccf_offline_stage1_test_revised.csv',keep_default_na = False)

In [3]:
#查看数据前6行
dfoff.head(10)
#dfon.head(6)
#dftest.head(6)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,null,null,0,null,20160217
1,1439408,4663,11002,150:20,1,20160528,null
2,1439408,2632,8591,20:1,0,20160217,null
3,1439408,2632,1078,20:1,0,20160319,null
4,1439408,2632,8591,20:1,0,20160613,null
5,1439408,2632,null,null,0,null,20160516
6,1439408,2632,8591,20:1,0,20160516,20160613
7,1832624,3381,7610,200:20,0,20160429,null
8,2029232,3381,11951,200:20,1,20160129,null
9,2029232,450,1532,30:5,0,20160530,null


In [8]:
shape_dfoff = dfoff.shape
print(shape_dfoff)
#做描述统计
print('有优惠券，购买商品: %d' % dfoff[(dfoff['Date_received'] != 'null') & (dfoff['Date'] != 'null')].shape[0])
print('有优惠卷，未购商品：%d' % dfoff[(dfoff['Date_received'] != 'null') & (dfoff['Date'] == 'null')].shape[0])
print('无优惠卷，购买商品：%d' % dfoff[(dfoff['Date_received'] == 'null') & (dfoff['Date'] != 'null')].shape[0])
print('无优惠卷，未购商品：%d' % dfoff[(dfoff['Date_received'] == 'null') & (dfoff['Date'] == 'null')].shape[0])


(1754884, 7)
有优惠券，购买商品: 75382
有优惠卷，未购商品：977900
无优惠卷，购买商品：701602
无优惠卷，未购商品：0


In [9]:
#特征提取
#首先考虑折扣
print('Discount_rate 类型：\n',dfoff['Discount_rate'].unique())
#可以提取4中特征：打折类型、折扣率、满多少、减多少

Discount_rate 类型：
 ['null' '150:20' '20:1' '200:20' '30:5' '50:10' '10:5' '100:10' '200:30'
 '20:5' '30:10' '50:5' '150:10' '100:30' '200:50' '100:50' '300:30'
 '50:20' '0.9' '10:1' '30:1' '0.95' '100:5' '5:1' '100:20' '0.8' '50:1'
 '200:10' '300:20' '100:1' '150:30' '300:50' '20:10' '0.85' '0.6' '150:50'
 '0.75' '0.5' '200:5' '0.7' '30:20' '300:10' '0.2' '50:30' '200:100'
 '150:5']


In [10]:
# 打折类型的转换
def getDiscountType(row):
    if row == 'null':
        return 'null'
    elif ':' in row:
        return 1
    else:
        return 0
#折扣率转换：没打折-1.0，满减-计算，折扣率-不变
def convertRate(row):
    if row == 'null':
        return 1.0
    elif ':' in row:
        rows = row.split(':')
        return 1.0 - float(rows[1])/float(rows[0])
    else:
        return float(row)
#满多少的转换
def getDiscountMan(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[0])
    else:
        return 0
#满多少的转换
def getDiscountJian(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[1])
    else:
        return 0
#应用定义的函数
def processData(df):
    df['discount_type'] = df['Discount_rate'].apply(getDiscountType)
    df['discount_rate'] = df['Discount_rate'].apply(convertRate)
    df['discount_man'] = df['Discount_rate'].apply(getDiscountMan)
    df['discount_jian'] = df['Discount_rate'].apply(getDiscountJian)
    print(df['discount_rate'].unique())
    return df


In [11]:
dfoff = processData(dfoff)
dftest = processData(dftest)

[1.         0.86666667 0.95       0.9        0.83333333 0.8
 0.5        0.85       0.75       0.66666667 0.93333333 0.7
 0.6        0.96666667 0.98       0.99       0.975      0.33333333
 0.2        0.4       ]
[0.83333333 0.9        0.96666667 0.8        0.95       0.75
 0.98       0.5        0.86666667 0.6        0.66666667 0.7
 0.85       0.33333333 0.94       0.93333333 0.975      0.99      ]
